In [9]:
!pip install fuzz
!pip install fuzzywuzzy
!pip install pandas as pd
!pip install json
!pip install csv
!pip install rdflib
!pip install folium
!pip install matplotlib
!pip install mapclassify

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


Lets create some code

#Read data and add to dataframe

In [64]:
from fuzzywuzzy import fuzz
import pandas as pd
from pandas import json_normalize
import json

# Read the CSV files
#file1 = pd.read_csv('Bochum_1_small.csv', encoding='latin-1')
file1 = pd.read_excel('Bochum_1_small.xlsx')


#file2 = pd.read_json('Mines_all.json')

with open('Mines_all_small.json') as f:
    data = json.load(f)
df = json_normalize(data)



# Read the JSON file
with open('Mines_all_small.json', 'r') as json_file:  # Replace 'your_file.json' with the path to your JSON file
    data = json.load(json_file)

# Filter the data to include only objects with the key "title"
titles = [location['title'] for location in data['locations']]
file2 = pd.DataFrame({"Title":titles})
#filtered_data = [entry for entry in data if 'title' in entry]

# Get the keys or titles from the filtered data
columns = list(titles)  # Assuming the filtered data is not empty

#print(titles)
print(file1.Mine)
print(file2.Title)

0     Bochumer Bergwerks-Actiengesellschaft (Zeche v...
1     Bochumer Bergwerks-Actiengesellschaft (Zeche v...
2     Bochumer Bergwerks-Actiengesellschaft (Zeche v...
3          Zeche Hasenwinkel bei Dahlhausen (a.d. Ruhr)
4                   Zeche ver. Maria-Anna und Steinbank
5                                 Zeche ver. Engelsburg
6     Gewerkschaft der Steinkohlenzeche ver. Carolin...
7               Gewerkschaft ver. Constantin der Grosse
8                   Aktiengesellschaft Zeche Dannenbaum
9                 Gewerkschaft der Zeche Julius Philipp
10      Gewerkschaft des Steinkohlenbergwerk Lothringen
11    Verwaltung der gewerkschaftlichen Mansfeldsche...
12                                 Gewerkschaft Berneck
13         Zeche Hasenwinkel bei Dahlhausen (a.d. Ruhr)
14                  Zeche ver. Maria-Anna und Steinbank
15                         Gewerkschaft ver. Engelsburg
16    Gewerkschaft der Steinkohlen- und Eisensteinze...
17              Gewerkschaft ver. Constantin der

In [7]:
file1.columns


file2['Title2']=file2['Title']
file2['Title2'] = file2['Title2'].str.replace(' Schacht', '', case=False)


In [15]:
df = json_normalize(data['locations'])
print(df)

                                                  text                  title  \
0    <b><a href="/wiki/index.php?title=Adele_Schach...   Adele Schacht Reiser   
1    <b><a href="/wiki/index.php?title=Adler_Schach...        Adler Schacht 1   
2    <b><a href="/wiki/index.php?title=Adler_Schach...        Adler Schacht 2   
3    <b><a href="/wiki/index.php?title=Adler_Schach...        Adler Schacht 3   
4    <b><a href="/wiki/index.php?title=Admiral_F%C3...  Admiral Förderschacht   
..                                                 ...                    ...   
865  <b><a href="/wiki/index.php?title=Zollverein_S...   Zollverein Schacht 7   
866  <b><a href="/wiki/index.php?title=Zollverein_S...   Zollverein Schacht 8   
867  <b><a href="/wiki/index.php?title=Zollverein_S...   Zollverein Schacht 9   
868  <b><a href="/wiki/index.php?title=Zweckel_Scha...      Zweckel Schacht 1   
869  <b><a href="/wiki/index.php?title=Zweckel_Scha...      Zweckel Schacht 2   

    link        lat       l

In [11]:
# Define a function to compute string similarity
def compute_similarity(string1, string2):
    return fuzz.ratio(string1, string2)  # You can use other similarity metrics here

# Iterate over rows in both files and compute similarity
matches = []
for index1, row1 in file1.iterrows():
    for index2, row2 in file2.iterrows():
        #print(row1['Mine'], row2['Title'])
        
        similarity_score = compute_similarity(row1['Mine-cleaned'], row2['Title2']) 
        #print(similarity_score)
        # Replace 'string_column' with the column name containing the strings
        if similarity_score >= 60:  # Set a threshold for similarity
            matches.append((row1['Mine'], row2['Title'], similarity_score))
            print(row1['Mine'], ['+'], row2['Title'])
            print(similarity_score)
            
# Output matches
if len(matches) == 0:
    print("No matches found.")
else:
    print("Matches found:")
    for match in matches:
        print(match)

# Write the filtered data to a CSV file
#with open('output.csv', 'w', newline='') as csv_file:  # Change 'output.csv' to the desired output file path
   # writer = csv.DictWriter(csv_file, fieldnames=columns)
    
    # Write the header
   # writer.writeheader()
    
    # Write the data
  #  for row in filtered_data:
    #    writer.writerow(row)

KeyError: 'Mine-cleaned'

In [19]:
print(df.title)

0       Adele Schacht Reiser
1            Adler Schacht 1
2            Adler Schacht 2
3            Adler Schacht 3
4      Admiral Förderschacht
               ...          
865     Zollverein Schacht 7
866     Zollverein Schacht 8
867     Zollverein Schacht 9
868        Zweckel Schacht 1
869        Zweckel Schacht 2
Name: title, Length: 870, dtype: object


In [25]:
# Create ttl file 
import pandas as pd
import json
from rdflib import Graph, Literal, Namespace, RDF, URIRef
from rdflib.namespace import GEO, XSD



# Define namespaces
EX = Namespace("http://example.org/")

# Create an RDF graph
g = Graph()
g.remove((None, None, None))
g.bind("ex", EX)
g.bind("geo", GEO)

# Process each entry in the JSON data
for index, row in df.iterrows():

    title_item = row.title
    latitude = row.lat
    longitude = row.lon
    
    # Create URI for the title and geometry
    title_uri = EX[title_item.replace(" ", "_")]
    geom_uri = EX[title_item.replace(" ", "_") + "Geom"]
    
    # Add RDF triples to the graph
    g.add((title_uri, RDF.type, EX.Schacht))
    g.add((title_uri, EX.name, Literal(title_item)))
    g.add((title_uri, GEO.hasGeometry, geom_uri))
    
    g.add((geom_uri, RDF.type, GEO.Geometry))
    g.add((geom_uri, GEO.asWKT, Literal(f"POINT({longitude} {latitude})", datatype=GEO.wktLiteral)))

# Serialize the graph to a TTL file
g.serialize(destination='Schaechte.ttl', format='turtle')

print("TTL file created successfully.")


TTL file created successfully.


In [29]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS

# Define namespaces
EX = Namespace("http://example.org/")
GEO = Namespace("http://www.opengis.net/ont/geosparql#")

# Load the RDF graph from the TTL file
g = Graph()
g.parse("Schaechte.ttl", format="turtle")

# Prepare a dictionary to hold the extracted data
data = []

# Iterate over all instances of ex:Schacht
for schacht in g.subjects(RDF.type, EX.Schacht):
    schacht_data = {
        "uri": str(schacht),
        "name": None,
        "geometry": None
    }
    
    # Get the name of the schacht
    for _, p, o in g.triples((schacht, EX.name, None)):
        if isinstance(o, Literal):
            schacht_data["name"] = str(o)
    
    # Get the geometry of the schacht
    for _, p, geom in g.triples((schacht, GEO.hasGeometry, None)):
        if isinstance(geom, URIRef):
            schacht_data["geometry"] = str(geom)
    
    data.append(schacht_data)

# Print the extracted data
for item in data:
    print(f"URI: {item['uri']}")
    print(f"Name: {item['name']}")
    print(f"Geometry: {item['geometry']}")
    print()

# Optionally, you can serialize the extracted data to JSON or any other format as needed
import json
with open("extracted_data.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

print("Data extraction complete.")


URI: http://example.org/Adele_Schacht_Reiser
Name: Adele Schacht Reiser
Geometry: http://example.org/Adele_Schacht_ReiserGeom

URI: http://example.org/Adler_Schacht_1
Name: Adler Schacht 1
Geometry: http://example.org/Adler_Schacht_1Geom

URI: http://example.org/Adler_Schacht_2
Name: Adler Schacht 2
Geometry: http://example.org/Adler_Schacht_2Geom

URI: http://example.org/Adler_Schacht_3
Name: Adler Schacht 3
Geometry: http://example.org/Adler_Schacht_3Geom

URI: http://example.org/Admiral_Förderschacht
Name: Admiral Förderschacht
Geometry: http://example.org/Admiral_FörderschachtGeom

URI: http://example.org/Admiral_Wetterschacht
Name: Admiral Wetterschacht
Geometry: http://example.org/Admiral_WetterschachtGeom

URI: http://example.org/Adolf_von_Hansemann_Ostschacht
Name: Adolf von Hansemann Ostschacht
Geometry: http://example.org/Adolf_von_Hansemann_OstschachtGeom

URI: http://example.org/Adolf_von_Hansemann_Schacht_1
Name: Adolf von Hansemann Schacht 1
Geometry: http://example.org/A

In [31]:
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, GEO

# Define namespaces
EX = Namespace("http://example.org/")
GEO = Namespace("http://www.opengis.net/ont/geosparql#")

# Load the RDF graph from the TTL file
g = Graph()
g.parse("Schaechte.ttl", format="turtle")

# Define the SPARQL query
query = """
PREFIX ex: <http://example.org/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?schacht ?name ?wkt
WHERE {
  ?schacht a ex:Schacht ;
           ex:name ?name ;
           geo:hasGeometry ?geom .
  ?geom geo:asWKT ?wkt .
}
"""

# Execute the query
results = g.query(query)

# Process and print the results
for row in results:
    print(f"Schacht URI: {row.schacht}")
    print(f"Name: {row.name}")
    print(f"Geometry (WKT): {row.wkt}")
    print()

# Optionally, save the results to a file
import json
data = [{"schacht": str(row.schacht), "name": str(row.name), "geometry": str(row.wkt)} for row in results]
with open("query_results.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

print("Query execution complete.")


Schacht URI: http://example.org/Adele_Schacht_Reiser
Name: Adele Schacht Reiser
Geometry (WKT): POINT(7.529716 51.491132)

Schacht URI: http://example.org/Adler_Schacht_1
Name: Adler Schacht 1
Geometry (WKT): POINT(7.087173 51.380766)

Schacht URI: http://example.org/Adler_Schacht_2
Name: Adler Schacht 2
Geometry (WKT): POINT(7.086953 51.378971)

Schacht URI: http://example.org/Adler_Schacht_3
Name: Adler Schacht 3
Geometry (WKT): POINT(7.084445 51.380472)

Schacht URI: http://example.org/Admiral_Förderschacht
Name: Admiral Förderschacht
Geometry (WKT): POINT(7.49893 51.468724)

Schacht URI: http://example.org/Admiral_Wetterschacht
Name: Admiral Wetterschacht
Geometry (WKT): POINT(7.498807 51.468099)

Schacht URI: http://example.org/Adolf_von_Hansemann_Ostschacht
Name: Adolf von Hansemann Ostschacht
Geometry (WKT): POINT(7.403537 51.578115)

Schacht URI: http://example.org/Adolf_von_Hansemann_Schacht_1
Name: Adolf von Hansemann Schacht 1
Geometry (WKT): POINT(7.376149 51.567835)

Schac

In [34]:
import geopandas as gpd
from shapely.geometry import Point
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, GEO

# Define namespaces
EX = Namespace("http://example.org/")
GEO = Namespace("http://www.opengis.net/ont/geosparql#")

# Load the RDF graph from the TTL file
g = Graph()
g.parse("Schaechte.ttl", format="turtle")

# Define the SPARQL query
query = """
PREFIX ex: <http://example.org/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?schacht ?name ?wkt
WHERE {
  ?schacht a ex:Schacht ;
           ex:name ?name ;
           geo:hasGeometry ?geom .
  ?geom geo:asWKT ?wkt .
}
"""

# Execute the query
results = g.query(query)

# Prepare data for GeoDataFrame
data = []
for row in results:
    schacht_uri = str(row.schacht)
    name = str(row.name)
    wkt = str(row.wkt)
    
    # Convert WKT to shapely geometry
    geometry = Point(float(wkt.split('(')[1].split()[0]), float(wkt.split('(')[1].split()[1][:-1]))
    
    data.append({"schacht_uri": schacht_uri, "name": name, "geometry": geometry})

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry="geometry")

# Save the GeoDataFrame to a shapefile
gdf.to_file("schachts.shp", driver='ESRI Shapefile')

print("Data extraction and shapefile creation complete.")


Data extraction and shapefile creation complete.


C:\Users\rohne\AppData\Local\Temp\ipykernel_41172\732286056.py:47: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("schachts.shp", driver='ESRI Shapefile')


In [47]:
import folium
import matplotlib
gdf = gdf.set_crs(epsg=4326)
gdf.explore()
m = folium.Map(location=[latitude, longitude], zoom_start=12)

# Add Choropleth layer to the map
folium.Geopandas(
    geo_data=gdf,
    data=gdf,
    columns=['geometry', 'name'],  # Specify the geometry and column to use
    key_on='feature.name',  # Key to link the GeoDataFrame and Choropleth
    fill_color='YlGnBu',  # Specify the color scheme
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='name'  # Specify the legend name
).add_to(m)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [66]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point



import pandas as pd
import geopandas as gpd
from shapely import wkt

# Read the specified sheet (Sheet 2) from the Excel file into a DataFrame
df = pd.read_excel("Result1_Zeche-Shaft-point.xlsx", sheet_name='Sheet2')  # You can also use sheet_name=1 to refer to the second sheet by index

df['POINT'] = df['POINT'].str.strip('"')

# Convert the 'POINT' column from string to Shapely Point objects
df['geometry'] = df['POINT'].apply(wkt.loads)  # Replace 'POINT' with the actual column name if different

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set the coordinate reference system (CRS) to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Use the explore method to visualize the data
# Color the points based on the 'Footprint' column (replace 'Footprint' with the actual column name)
#gdf.explore(column='Zeche', cmap='viridis', legend=True)
gdf.explore()
# Set the coordinate reference system (CRS) to
